# Inference

Pylogic is used to write out proofs to be checked. This means you must already know the proof of what you want to show, as there is currently no proof-search functionality.

The way to write proofs in Pylogic is:

1. (Optional but helpful) Define the statement you want to prove. Do not assume it.
2. If you want to prove an implication or a quantified sentence, create an assumption context. Else, skip this step.
3. Create your assumptions/premises and assume them.
4. Call different methods on your assumptions to prove intermediate results.
5. Call different methods on intermediate results to prove the final result.
6. Check that the final result is what you wanted to prove. You can use the `has_been_proven` function to check this.

# Example 1

We showcase this process in the following example. Given the premises $P$, $P \implies (Q \lor R)$ and $(Q \lor R) \implies \neg S$, we want to prove $\neg S$.

We already know the proof: apply modus ponens twice. We will write this proof in Pylogic.

In [2]:
from pylogic import *

In [3]:
P, Q, R, S = propositions("P", "Q", "R", "S")
to_prove = neg(S) # what we want to prove

In [4]:
# premises
P.assume() # P is assumed
P_implies_Q_or_R = assume(P.implies(Q.or_(R))) # P => (Q or R) is assumed
Q_or_R_implies_not_S = assume(Q.or_(R).implies(neg(S))) # (Q or R) => ~S is assumed

display(P, P_implies_Q_or_R, Q_or_R_implies_not_S)

Proposition(P)

Implies(Proposition(P), Or(Proposition(Q), Proposition(R)))

Implies(Or(Proposition(Q), Proposition(R)), Not(Proposition(S)))

Now we actually write the proof. We check intermediate results to make sure we are on the right track.

In [5]:
Q_or_R = P.modus_ponens(P_implies_Q_or_R) # since P is true and P => (Q or R) is true, Q or R must be true (modus ponens)
display(Q_or_R, Q_or_R.is_proven)

Or(Proposition(Q), Proposition(R))

True

In [6]:
# since Q or R is true and (Q or R) => ~S is true, ~S must be true (modus ponens)
not_S = Q_or_R.modus_ponens(Q_or_R_implies_not_S)
display(not_S, not_S.is_proven)

Not(Proposition(S))

True

We are done at this point. We can check if the final result is what we wanted to prove.

In [7]:
has_been_proven(to_prove, not_S) # check if the proof is correct

True

Clearly, the most important aspect of this is knowing which methods to call. You can see a list of all inference-rule methods that can be called on a proposition by calling the `inference_rules` method on it. For instance:

In [8]:
P.inference_rules()

['assume',
 'substitute',
 'by_inspection',
 'and_',
 'or_',
 'p_and_reverse',
 'modus_ponens',
 'mp',
 'modus_tollens',
 'mt',
 'is_one_of',
 'is_special_case_of',
 'thus_there_exists',
 'thus_forall',
 'contradicts']

In this example, we knew that we needed to use modus ponens. Therefore, checking the `inference_rules` method and seeing that it has a `modus_ponens` method is useful. We can then look at the help message for `modus_ponens` to see how to use it.

In [9]:
help(P.modus_ponens)

Help on method modus_ponens in module pylogic.proposition.proposition:

modus_ponens(other: 'Implies[Self, TProposition]') -> 'TProposition' method of pylogic.proposition.proposition.Proposition instance
    Logical inference rule.
    other: Implies
        Must be an implication that has been proven whose structure is
        self -> OtherProposition
    
    Examples
    --------
    >>> p1 = prop("P").assume() # P
    >>> p2 = prop("P").implies(prop("Q")).assume() # P -> Q
    >>> p3 = p1.modus_ponens(p2) # infer Q
    >>> p3, p3.is_proven
    (Q, True)



# Example 2: By Cases and AssumptionsContext

Here, we perform a proof by cases. Given the premises 
1. $P \lor Q$
2. $P \implies A$ 
3. $Q \implies B$
4. $A \implies R$
5. $B \implies R$

we want to prove $R$. Since $P \lor Q$ is a disjunction, we can prove $R$ by proving $R$ under both cases $P$ and $Q$.

In Pylogic, we do this by proving the implications $P \implies R$ and $Q \implies R$ separately. We then use the `by_cases` method on the disjunction to prove $R$ conclusively.

In [10]:
P, Q, R, A, B = propositions("P", "Q", "R", "A", "B")
to_prove = R

# premises
P_or_Q = assume(P.or_(Q))
P_implies_A = assume(P.implies(A))
Q_implies_B = assume(Q.implies(B))
A_implies_R = assume(A.implies(R))
B_implies_R = assume(B.implies(R))

In [11]:
# case 1: P is true
with AssumptionsContext() as case1:
    P.assume()
    R = P.modus_ponens(P_implies_A).modus_ponens(A_implies_R)
    conclude(R)
P_implies_R = case1.get_proven()[0]

# case 2: Q is true
with AssumptionsContext() as case2:
    Q.assume()
    R = Q.modus_ponens(Q_implies_B).modus_ponens(B_implies_R)
    conclude(R)
Q_implies_R = case2.get_proven()[0]

# since P or Q is true and (P => R) and (Q => R) are true, R must be true
R_proven = P_or_Q.by_cases(P_implies_R, Q_implies_R)
display(R_proven)
has_been_proven(to_prove, R_proven) # check if the proof is correct

Proposition(R)

True

In the example above, we see a usage of the `get_proven` method of an assumptions context. When the context is closed, we can use this method to get the implications that have been proven by assuming propositions in the context and concluding other propositions. This is the main method of proving implications in Pylogic.

In order to know what implications to prove when the context is closed, we need to call the `conclude` function with any conclusions made within the context. The next example shows more of this.

## Example 3: Proving Implications

To prove an implication, assume the antecedent in a context and try to prove the consequent.

In this example, we are given the premises

1. $P \implies Q$
2. $Q \implies R$
3. $R \implies S$

and we want to prove $\neg S \implies \neg P$.

In [12]:
P, Q, R, S = propositions("P", "Q", "R", "S")
to_prove = neg(S).implies(neg(P)) 

# premises
P_implies_Q, Q_implies_R, R_implies_S = assume(P.implies(Q), Q.implies(R), R.implies(S))

with AssumptionsContext() as ctx:
    not_S = neg(S).assume()
    not_P = not_S.modus_tollens(R_implies_S).modus_tollens(Q_implies_R).modus_tollens(P_implies_Q)
    conclude(not_P)
print(ctx.get_proven())

not_S_implies_not_P = ctx.get_proven()[0]
display(not_S_implies_not_P, not_S_implies_not_P.is_proven)
has_been_proven(to_prove, not_S_implies_not_P) # check if the proof is correct

[Implies(Not(Proposition(S)), Not(Proposition(P)))]


Implies(Not(Proposition(S)), Not(Proposition(P)))

True

True

## AssumptionsContext and Implications (and Universally Quantified Sentences)

This was discussed in the previous section, but one thing to keep in mind is that `AssumptionsContext` is useful for proving implications. To prove $P \implies Q$, for instance, your code will look something like:

```python
with AssumptionsContext() as ctx:
    assume(P)
    ... # rules of inference are applied to deduce Q
    conclude(Q)

p_implies_q = ctx.get_proven()[0]
print(p_implies_q) # P -> Q
print(p_implies_q.is_proven) # True
```

The `conclude` function is used to tell Pylogic that, when the context is exited, the implication "assumptions implies conclusion" is proven.
Compare this to a written proof. To prove $P \implies Q$, you would write something like:

1. Assume $P$.
2. ... (some steps)
3. Therefore, $Q$.
4. Therefore, $P \implies Q$.

The `get_proven` method returns a list of all implications that have been proven as a result of the context. In this case, there is only one: $P \implies Q$.

In the next example, we will see how to prove universally quantified sentences.

## Example 4: Proving Universally Quantified Sentences

To prove a `Forall` sentence, create a variable **within a context** and try to prove the sentence for that variable. The universally quantified sentence is then proven at the end of the context.

To create a variable within a context, use the `context_variables` (or `ctx_vars`) function which is like the `variables` function but for contexts.

In this example, we are given the premises

1. $\forall x (P(x) \implies Q(x))$
2. $\forall x (Q(x) \implies R(x))$
3. $\forall x \neg R(x)$

and we want to prove $\forall x \neg P(x)$.

In [13]:
x = Variable("x")
Px, Qx, Rx = propositions("P", "Q", "R", args=(x,))

# premises
p1 = Forall(x, Px.implies(Qx)).assume()
p2 = Forall(x, Qx.implies(Rx)).assume()
p3 = Forall(x, neg(Rx)).assume()

with AssumptionsContext() as ctx:
    x = context_variables("x")
    not_Rx = p3(x) # universal instantiation
    not_Qx = not_Rx.modus_tollens(p2(x))
    not_Px = not_Qx.modus_tollens(p1(x))

    conclude(not_Px)

forall_x_not_Px = ctx.get_proven()[0]
display(forall_x_not_Px)

Forall(Variable(x, deps=()), Not(Proposition(P, x)))

We see some new features in this example:

1. The `context_variables` function is used to create a variable within a context.
2. `Forall` propositions are like functions. You can call it with a term, if it has been proven, and it will return the proposition with the term substituted in. This is the "Universal Instantiation" rule of inference. You can look up this rule online.
3. The `conclude` function is used to tell Pylogic that, when the context is exited, the universally quantified sentence is proven.

Since a variable was created in the context, the context will create a universally quantified sentence when exited. This is the main way to prove universally quantified sentences in Pylogic.

Compare the Pylogic proof to a written proof:

1. We are given $\forall x (P(x) \implies Q(x))$, $\forall x (Q(x) \implies R(x))$, and $\forall x \neg R(x)$.
2. Let $x$ be arbitrary.
3. Then $\neg R(x)$, $Q(x) \implies R(x)$, and $P(x) \implies Q(x)$ (universal instantiation).
4. Then $\neg Q(x)$ by modus tollens.
5. Then $\neg P(x)$ by modus tollens.
6. Since $x$ was arbitrary, $\forall x \neg P(x)$.

## Conclusion

It takes some practice to get used to writing proofs in Pylogic. The key is to know what you want to prove and to know which methods to call on propositions to prove them. The `inference_rules` method is useful for this.

[Previous: Assumptions](Assumptions.ipynb)

## Contents

1. [Introduction](Introduction.ipynb)
2. [Terms](Terms.ipynb)
3. [Expressions](Expressions.ipynb)
4. [Propositions](Propositions.ipynb)
5. [Compound Propositions](Compound%20Propositions.ipynb)
6. [Relations](Relations.ipynb)
7. [Quantifiers](Quantifiers.ipynb)
8. [Assumptions](Assumptions.ipynb)
9. [Inference](#)